### urllib

In [2]:
import urllib

In [3]:
urllib??

urllib.request

urllib.error

urllib.parse

urllib.robotparser

urllib.response

In [4]:
from urllib import robotparser

robot = robotparser.RobotFileParser()
robot.set_url("https://www.google.com/robots.txt")
robot.read()
robot.can_fetch("Bot", "/search/about")

False

In [5]:
robot = robotparser.RobotFileParser()
robot.set_url("https://news.naver.com/robots.txt")
robot.read()
robot.can_fetch("Yeti", "/main/imagemontage")

True

In [6]:
robot = robotparser.RobotFileParser()
robot.set_url("http://gsen.hanyang.ac.kr/robots.txt")
robot.read()
robot.can_fetch("*", "/admin")

True

In [7]:
from urllib import request

In [12]:
resp = request.urlopen('https://www.google.com')

In [13]:
resp.geturl()

'https://www.google.com'

In [14]:
resp.reason

'OK'

In [15]:
resp.getcode()

200

In [17]:
resp.getheaders()

[('Date', 'Thu, 11 Jul 2019 02:10:13 GMT'),
 ('Expires', '-1'),
 ('Cache-Control', 'private, max-age=0'),
 ('Content-Type', 'text/html; charset=ISO-8859-1'),
 ('P3P', 'CP="This is not a P3P policy! See g.co/p3phelp for more info."'),
 ('Server', 'gws'),
 ('X-XSS-Protection', '0'),
 ('X-Frame-Options', 'SAMEORIGIN'),
 ('Set-Cookie',
  '1P_JAR=2019-07-11-02; expires=Sat, 10-Aug-2019 02:10:13 GMT; path=/; domain=.google.com'),
 ('Set-Cookie',
  'NID=187=Yw2_M_mhK_xcamR5PyWDhMVnuF5lt3Zw8nw7sEwnVdII3rCYo_VkEE1ZYHsgtkNGOe7alRpzBVOvAr5YSn9l7Il5PUfANsj9TDkFWTKDijjVatKhLHrvrJHLFaWj8dEu1xG7-Jd4V4CAESB6GoJjYztcCd4EC2DKI5HT-wrGQsY; expires=Fri, 10-Jan-2020 02:10:13 GMT; path=/; domain=.google.com; HttpOnly'),
 ('Alt-Svc', 'quic=":443"; ma=2592000; v="46,43,39"'),
 ('Accept-Ranges', 'none'),
 ('Vary', 'Accept-Encoding'),
 ('Connection', 'close')]

In [18]:
resp.status

200

In [19]:
resp.info()

https://www.google.com/search?

    q=%EC%B0%A8%EC%9D%80%EC%9A%B0&
    
    oq=%EC%B0%A8%EC%9D%80%EC%9A%B0&
    
    aqs=chrome..69i57j0l5.1862j0j8&
    
    sourceid=chrome&
    
    ie=UTF-8

https://www.google.com/search?q=%EC%B0%A8%EC%9D%80%EC%9A%B0&

In [25]:
resp = request.urlopen("https://www.google.com/search?q=%EC%B0%A8%EC%9D%80%EC%9A%B0&")
resp.code, resp.reason                        

HTTPError: HTTP Error 403: Forbidden

-> 정상적인 url이 아니라 봇이라고 판단하고 차단된다.

에러 핸들링

In [26]:
from urllib import error

try:
    resp = request.urlopen("https://www.google.com/search?q=%EC%B0%A8%EC%9D%80%EC%9A%B0&")
except error.HTTPError as e:
    print(e.code, e.reason, e.headers)

403 Forbidden Content-Type: text/html; charset=UTF-8
Date: Thu, 11 Jul 2019 02:33:34 GMT
Server: gws
Cache-Control: private
X-XSS-Protection: 0
X-Frame-Options: SAMEORIGIN
Alt-Svc: quic=":443"; ma=2592000; v="46,43,39"
Accept-Ranges: none
Vary: Accept-Encoding
Connection: close




봇인걸 숨겨보자

내 User-Agent를 알아보자

Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36

In [33]:
headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}

In [31]:
request.urlopen??

-> 핸들링 할 수 있는게 없다.

In [30]:
request.Request??

In [35]:
try:
    req = request.Request("https://www.google.com/search?q=%EC%B0%A8%EC%9D%80%EC%9A%B0&",
                          headers=headers)
    resp = request.urlopen(req)
except error.HTTPError as e:
    print(e.code, e.reason, e.headers)

In [40]:
resp.code, resp.reason, req.headers

(200,
 'OK',
 {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'})

In [41]:
try:
    req = request.Request("https://www.google.com/search?q=차은우",
                          headers=headers)
    resp = request.urlopen(req)
except error.HTTPError as e:
    print(e.code, e.reason, e.headers)

UnicodeEncodeError: 'ascii' codec can't encode characters in position 14-16: ordinal not in range(128)

통신은 바이트코드로..

상대 경로일때 : parse.urljoin

유니코드(한글) -> 헥사데시말 : parse.urlencode

In [42]:
from urllib import parse

In [45]:
parse.urlparse('https://www.google.com/search?q=%EC%B0%A8%EC%9D%80%EC%9A%B0&')

ParseResult(scheme='https', netloc='www.google.com', path='/search', params='', query='q=%EC%B0%A8%EC%9D%80%EC%9A%B0&', fragment='')

In [46]:
parse.urljoin('https://www.google.com/search?q=%EC%B0%A8%EC%9D%80%EC%9A%B0&','/search/about')

'https://www.google.com/search/about'

In [47]:
parse.urlencode({'q':'파이썬'})

'q=%ED%8C%8C%EC%9D%B4%EC%8D%AC'

In [48]:
parse.quote_plus('파이썬')

'%ED%8C%8C%EC%9D%B4%EC%8D%AC'

In [50]:
headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}
params = {}
try:
    req = request.Request("https://www.google.com/search?q=%EC%B0%A8%EC%9D%80%EC%9A%B0&",
                          headers=headers)
    resp = request.urlopen(req)
except error.HTTPError as e:
    print(e.code, e.reason, e.headers)

In [51]:
resp.code, resp.reason, req.headers

(200,
 'OK',
 {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'})

---

### Requests

request.get/.post/.put/.delete/.head/.options/.*response

In [69]:
import requests

In [54]:
requests.get??

In [101]:
requests.request??

httpbin.org

http://httpbin.org/get?key=value

In [73]:
headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}

resp = requests.request("get", 
                        "http://httpbin.org/", 
                        params={"key":"value"})

In [74]:
print(resp.text)

<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <title>httpbin.org</title>
    <link href="https://fonts.googleapis.com/css?family=Open+Sans:400,700|Source+Code+Pro:300,600|Titillium+Web:400,600,700"
        rel="stylesheet">
    <link rel="stylesheet" type="text/css" href="/flasgger_static/swagger-ui.css">
    <link rel="icon" type="image/png" href="/static/favicon.ico" sizes="64x64 32x32 16x16" />
    <style>
        html {
            box-sizing: border-box;
            overflow: -moz-scrollbars-vertical;
            overflow-y: scroll;
        }

        *,
        *:before,
        *:after {
            box-sizing: inherit;
        }

        body {
            margin: 0;
            background: #fafafa;
        }
    </style>
</head>

<body>
    <a href="https://github.com/requests/httpbin" class="github-corner" aria-label="View source on Github">
        <svg width="80" height="80" viewBox="0 0 250 250" style="fill:#151513; color:#fff; position: absolute;

In [75]:
resp = requests.request("post", "http://httpbin.org/post", data={"key":"value"})

In [76]:
print(resp.text)

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "key": "value"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "9", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.21.0"
  }, 
  "json": null, 
  "origin": "163.152.3.131, 163.152.3.131", 
  "url": "https://httpbin.org/post"
}



error handling ->  urlib vs requsts

In [77]:
import requests

In [ ]:
headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}

def download(method, url, param=None, data=None):
    try:
        resp = requests.request(method, url,
                         params, data,
                         headers=headers)
    except requests.exceptions.HTTPError as e:
        print(e.code)
        print(e.reson)

In [119]:
headers = {}
params = {}

In [120]:
def download(method, url, param=None, data=None):
    try:
        resp = requests.request(method, url,
                         params=params, data=data,
                         headers=None)
    except requests.exceptions.HTTPError as e:
        print(e.response)
        print(e.reson)
    return resp

In [121]:
headers = None

In [122]:
html = download("get","https://www.google.com/search?p=차은우")

In [123]:
html.text

'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="ko"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="+KaCTZzGrUyMvQVQEPMWRQ==">(function(){window.google={kEI:\'McImXZmPCZDS8wWB_azoAQ\',kEXPI:\'0,1353804,1958,1640,782,1225,590,141,223,510,18,228,819,3151,379,206,904,113,54,122,1027,79,202,53,187,55,6,323,78,10,73,1133444,1197894,329517,1294,12383,4855,32691,15248,867,12163,16521,363,8825,2436,266,5110,572,835,284,2,204,374,728,2069,362,1362,4323,4968,773,2254,4740,3122,2591,3601,669,1050,1808,1397,81,7,491,2044,8909,4604,693,796,1220,38,920,746,7,120,1217,1364,346,1,1264,2736,48,3013,2,631,3241,43,1851,677,2255,2607,632,1139,1089,655,21,317,1118,475,427,104,86,2,1159,777,1,369,1316,705,756,98,392,29,400,992,1107,10,168,8,109,1018,140,1356,1140,48,553,25,10,1269,1683,529,25,177,167,156,5,1245,7,840,324,193,

In [124]:
html.headers

{'Date': 'Thu, 11 Jul 2019 04:59:29 GMT', 'Expires': '-1', 'Cache-Control': 'private, max-age=0', 'Content-Type': 'text/html; charset=ISO-8859-1', 'P3P': 'CP="This is not a P3P policy! See g.co/p3phelp for more info."', 'Content-Encoding': 'gzip', 'Server': 'gws', 'X-XSS-Protection': '0', 'X-Frame-Options': 'SAMEORIGIN', 'Set-Cookie': '1P_JAR=2019-07-11-04; expires=Sat, 10-Aug-2019 04:59:29 GMT; path=/; domain=.google.com, NID=187=NnvufBoxTmvA6HEWtUuO_E7jnzdN3msuFyzg3F7VvVHHxSW4YnM58fzsiOhZWvU7wG6rJh1mKghSSXwwf1Oy6dbr3Wonv9z2m0OYrKRVj1WpYoky93ORcmroqiNU3i4Nzo5OCPMix4ZQxFSJoxY9agVqavwHi9Y4_Z4YAxT9DVI; expires=Fri, 10-Jan-2020 04:59:29 GMT; path=/; domain=.google.com; HttpOnly', 'Alt-Svc': 'quic=":443"; ma=2592000; v="46,43,39"', 'Transfer-Encoding': 'chunked'}

In [125]:
html.reason

'OK'

In [126]:
html.status_code

200

In [127]:
params

{}

https://www.crawler-test.com/status_codes/status_403

In [131]:
def download(method, url, param=None, data=None):
    try:
        resp = requests.request(method, url,
                         params=params, data=data,
                         headers=None)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        print(e.response.status_code)
        print(e.response.reason)
    return resp

In [132]:
html = download("get","https://www.crawler-test.com/status_codes/status_403")

403
Forbidden


In [133]:
html = download("get","https://www.google.com/search?p=차은우")

In [137]:
import time

def download(method, url, param=None, data=None):
    try:
        resp = requests.request(method, url,
                         params=params, data=data,
                         headers=None)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600:
            time.sleep(1)
            print("재시도")
        else:
            print(e.response.status_code)
            print(e.response.reason)       
    return resp

In [138]:
html = download("get","https://www.crawler-test.com/status_codes/status_500")

재시도


In [136]:
html = download("get","https://www.crawler-test.com/status_codes/status_403")

403
Forbidden


In [139]:
time.sleep??

In [149]:
import time

def download(method, url, param=None, data=None):
    try:
        resp = requests.request(method, url,
                         params=params, data=data,
                         headers=None)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600\
            and maxretries > 0:
            print(maxretries)
            time.sleep(timeout)
            resp = download(method, url, param, data, timeout, maxretries-1)
            print("재시도")
        else:
            print(e.response.status_code)
            print(e.response.reason)       
    return resp

In [146]:
html = download("get","https://www.crawler-test.com/status_codes/status_403")

403
Forbidden


In [157]:
html = download("get", "http://httpbin.org/get", {"key":"value"})
html.text

'{\n  "args": {}, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Host": "httpbin.org", \n    "User-Agent": "python-requests/2.21.0"\n  }, \n  "origin": "163.152.3.131, 163.152.3.131", \n  "url": "https://httpbin.org/get"\n}\n'

In [184]:
import json

In [199]:
url = 'http://openapi.airkorea.or.kr/openapi/services/rest/ArpltnInforInqireSvc/getCtprvnMesureSidoLIst'
params = {
    "serviceKey": "PGlojt4q3MXsNb2WywMWZNXPnV0sMYmPnZHjdER8uUq0ceI9rGlicEnKMGOoGnFeTGeEZZI9MeJAWZ43cXokzw%3D%3D",
    "numOfRows":10,
    "pageNo":1,
    "sidoName":"서울",
    "searchCondition":"DAILY",
    "_returnType":"JSON"
}
html = download("get",
                url, 
               params)

In [200]:
html.text

'<?xml version="1.0" encoding="UTF-8"?>\r\n\r\n\r\n\r\n\r\n<response>\r\n\t<header>\r\n\t\t<resultCode>30</resultCode>\r\n\t\t<resultMsg>SERVICE KEY IS NOT REGISTERED ERROR.</resultMsg>\r\n\t</header>\r\n</response>\r\n'

params["serviceKey"] = urllib.parse.unquote(params["serviceKey"])

servicekey

In [192]:
html.encoding

'utf-8'

In [193]:
html.url

'http://openapi.airkorea.or.kr/openapi/services/rest/ArpltnInforInqireSvc/getCtprvnMesureSidoLIst'

In [208]:
requests.compat.unquote??

In [203]:
params

{'serviceKey': 'PGlojt4q3MXsNb2WywMWZNXPnV0sMYmPnZHjdER8uUq0ceI9rGlicEnKMGOoGnFeTGeEZZI9MeJAWZ43cXokzw%3D%3D',
 'numOfRows': 10,
 'pageNo': 1,
 'sidoName': '서울',
 'searchCondition': 'DAILY',
 '_returnType': 'JSON'}

In [204]:
params["serviceKey"] = requests.compat.unquote(params["serviceKey"])

In [207]:
params

{'serviceKey': 'PGlojt4q3MXsNb2WywMWZNXPnV0sMYmPnZHjdER8uUq0ceI9rGlicEnKMGOoGnFeTGeEZZI9MeJAWZ43cXokzw==',
 'numOfRows': 10,
 'pageNo': 1,
 'sidoName': '서울',
 'searchCondition': 'DAILY',
 '_returnType': 'JSON'}

In [205]:
html = download("get",
                url, 
               params)

In [206]:
html.text

'{"list":[{"_returnType":"json","cityName":"강남구","cityNameEng":"Gangnam-gu","coValue":"0.2","dataGubun":"","dataTime":"2019-07-11 14:00","districtCode":"02","districtNumSeq":"001","itemCode":"","khaiValue":"","no2Value":"0.011","numOfRows":"10","o3Value":"0.025","pageNo":"1","pm10Value":"9","pm25Value":"4","resultCode":"","resultMsg":"","searchCondition":"","serviceKey":"","sidoName":"서울","so2Value":"0.005","totalCount":""},{"_returnType":"json","cityName":"강동구","cityNameEng":"Gangdong-gu","coValue":"0.3","dataGubun":"","dataTime":"2019-07-11 14:00","districtCode":"02","districtNumSeq":"002","itemCode":"","khaiValue":"","no2Value":"0.008","numOfRows":"10","o3Value":"0.029","pageNo":"1","pm10Value":"6","pm25Value":"2","resultCode":"","resultMsg":"","searchCondition":"","serviceKey":"","sidoName":"서울","so2Value":"0.003","totalCount":""},{"_returnType":"json","cityName":"강북구","cityNameEng":"Gangbuk-gu","coValue":"0.2","dataGubun":"","dataTime":"2019-07-11 14:00","districtCode":"02","distr

In [209]:
obj = json.loads(html.text)

In [210]:
obj

{'list': [{'_returnType': 'json',
   'cityName': '강남구',
   'cityNameEng': 'Gangnam-gu',
   'coValue': '0.2',
   'dataGubun': '',
   'dataTime': '2019-07-11 14:00',
   'districtCode': '02',
   'districtNumSeq': '001',
   'itemCode': '',
   'khaiValue': '',
   'no2Value': '0.011',
   'numOfRows': '10',
   'o3Value': '0.025',
   'pageNo': '1',
   'pm10Value': '9',
   'pm25Value': '4',
   'resultCode': '',
   'resultMsg': '',
   'searchCondition': '',
   'serviceKey': '',
   'sidoName': '서울',
   'so2Value': '0.005',
   'totalCount': ''},
  {'_returnType': 'json',
   'cityName': '강동구',
   'cityNameEng': 'Gangdong-gu',
   'coValue': '0.3',
   'dataGubun': '',
   'dataTime': '2019-07-11 14:00',
   'districtCode': '02',
   'districtNumSeq': '002',
   'itemCode': '',
   'khaiValue': '',
   'no2Value': '0.008',
   'numOfRows': '10',
   'o3Value': '0.029',
   'pageNo': '1',
   'pm10Value': '6',
   'pm25Value': '2',
   'resultCode': '',
   'resultMsg': '',
   'searchCondition': '',
   'serviceKey'

In [221]:
html = download("get",
                "https://www.naver.com", 
               {})

In [222]:
html

<Response [200]>

In [223]:
html.text

'<!doctype html>\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n<html lang="ko">\n<head>\n<meta charset="utf-8">\n<meta name="Referrer" content="origin">\n<meta http-equiv="Content-Script-Type" content="text/javascript">\n<meta http-equiv="Content-Style-Type" content="text/css">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="viewport" content="width=1100">\n<meta name="apple-mobile-web-app-title" content="NAVER" />\n<meta name="robots" content="index,nofollow"/>\n<meta name="description" content="네이버 메인에서 다양한 정보와 유용한 컨텐츠를 만나 보세요"/>\n<meta property="og:title" content="네이버">\n<meta property="og:url" content="https://www.naver.com/">\n<meta property="og:image" content="https://s.pstatic.net/static/www/mobile/edit/2016/0705/mobile_212852414260.png">\n<meta property="og:description" content="네이버 메인에서 다양한 정보와 유용한 컨텐츠를 만나 보세요"/>\n<meta name="twitter:card" content="summary">\n<meta name="twitter:title" content="">\n<meta name="twitter:url" content="https://www.naver.com/">\n<meta name="

In [214]:
from html import unescape

In [217]:
html.encoding = "utf-8"

In [218]:
html.text

'<!doctype html>\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n<html lang="ko">\n<head>\n<meta charset="utf-8">\n<meta name="Referrer" content="origin">\n<meta http-equiv="Content-Script-Type" content="text/javascript">\n<meta http-equiv="Content-Style-Type" content="text/css">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="viewport" content="width=1100">\n<meta name="apple-mobile-web-app-title" content="NAVER" />\n<meta name="robots" content="index,nofollow"/>\n<meta name="description" content="네이버 메인에서 다양한 정보와 유용한 컨텐츠를 만나 보세요"/>\n<meta property="og:title" content="네이버">\n<meta property="og:url" content="https://www.naver.com/">\n<meta property="og:image" content="https://s.pstatic.net/static/www/mobile/edit/2016/0705/mobile_212852414260.png">\n<meta property="og:description" content="네이버 메인에서 다양한 정보와 유용한 컨텐츠를 만나 보세요"/>\n<meta name="twitter:card" content="summary">\n<meta name="twitter:title" content="">\n<meta name="twitter:url" content="https://www.naver.com/">\n<meta name="

In [224]:
unescape(html.text)

'<!doctype html>\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n<html lang="ko">\n<head>\n<meta charset="utf-8">\n<meta name="Referrer" content="origin">\n<meta http-equiv="Content-Script-Type" content="text/javascript">\n<meta http-equiv="Content-Style-Type" content="text/css">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="viewport" content="width=1100">\n<meta name="apple-mobile-web-app-title" content="NAVER" />\n<meta name="robots" content="index,nofollow"/>\n<meta name="description" content="네이버 메인에서 다양한 정보와 유용한 컨텐츠를 만나 보세요"/>\n<meta property="og:title" content="네이버">\n<meta property="og:url" content="https://www.naver.com/">\n<meta property="og:image" content="https://s.pstatic.net/static/www/mobile/edit/2016/0705/mobile_212852414260.png">\n<meta property="og:description" content="네이버 메인에서 다양한 정보와 유용한 컨텐츠를 만나 보세요"/>\n<meta name="twitter:card" content="summary">\n<meta name="twitter:title" content="">\n<meta name="twitter:url" content="https://www.naver.com/">\n<meta name="

In [230]:
unescape??

In [226]:
html = download("post",
                "http://www.kyobobook.co.kr/search/SearchCommonMain.jsp?vPstrCategory=TOT&vPstrKeyWord=%26%2352264%3B%26%2351008%3B%26%2350864%3B&vPplace=top", 
               {"vPstrKeyWord":"차은우"})

In [227]:
html.text

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml">\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n<!-- header -->\n \n\n\n\n\n\n\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\n\n\n<head>\n\t<title>&#52264;&#51008;&#50864; -  인터넷교보문고</title>\n<META http-equiv="Pragma" content="nocache">\n<meta http-equiv="Expires" content="0"/>\n<META http-equiv="Cache-Control" content="no-cache">\n<META http-equiv=\'Content-Type\' content=\'text/html; charset=euc-kr\'>\n<meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1">\n<!--<LINK href="/common/Style.css" rel="stylesheet" type="text/css"/>-->\n<link rel="shortcut icon" href="/newimages/apps/b2c/kyobo.ICO"/>\n<link rel="stylesheet" type="text/css" href="http://image.kyobobook.co.kr/ink/css/default_ink.css" />\n<link rel="stylesheet" type="text/css" href="http://image.kyobobook.co.kr/ink/css/search.css" />\n<!--\n

In [228]:
html.status_code

200

In [229]:
unescape(html.text)

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml">\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n<!-- header -->\n \n\n\n\n\n\n\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\n\n\n<head>\n\t<title>차은우 -  인터넷교보문고</title>\n<META http-equiv="Pragma" content="nocache">\n<meta http-equiv="Expires" content="0"/>\n<META http-equiv="Cache-Control" content="no-cache">\n<META http-equiv=\'Content-Type\' content=\'text/html; charset=euc-kr\'>\n<meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1">\n<!--<LINK href="/common/Style.css" rel="stylesheet" type="text/css"/>-->\n<link rel="shortcut icon" href="/newimages/apps/b2c/kyobo.ICO"/>\n<link rel="stylesheet" type="text/css" href="http://image.kyobobook.co.kr/ink/css/default_ink.css" />\n<link rel="stylesheet" type="text/css" href="http://image.kyobobook.co.kr/ink/css/search.css" />\n<!--\n<link rel="stylesheet

In [231]:
html.url

'http://www.kyobobook.co.kr/search/SearchCommonMain.jsp?vPstrCategory=TOT&vPstrKeyWord=%26%2352264%3B%26%2351008%3B%26%2350864%3B&vPplace=top&serviceKey=PGlojt4q3MXsNb2WywMWZNXPnV0sMYmPnZHjdER8uUq0ceI9rGlicEnKMGOoGnFeTGeEZZI9MeJAWZ43cXokzw%3D%3D&numOfRows=10&pageNo=1&sidoName=%EC%84%9C%EC%9A%B8&searchCondition=DAILY&_returnType=JSON'

In [250]:
html2 = download("post",
                "http://www.kyobobook.co.kr/search/SearchCommonMain.jsp?vPstrCategory=TOT&vPstrKeyWord=%26%2352264%3B%26%2351008%3B%26%2350864%3B&vPplace=top", 
               {"vPstrKeyWord":"python",
               "vPchgCategory":"TOT"})

In [252]:
html2.text

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml">\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n<!-- header -->\n \n\n\n\n\n\n\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\n\n\n<head>\n\t<title>&#52264;&#51008;&#50864; -  인터넷교보문고</title>\n<META http-equiv="Pragma" content="nocache">\n<meta http-equiv="Expires" content="0"/>\n<META http-equiv="Cache-Control" content="no-cache">\n<META http-equiv=\'Content-Type\' content=\'text/html; charset=euc-kr\'>\n<meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1">\n<!--<LINK href="/common/Style.css" rel="stylesheet" type="text/css"/>-->\n<link rel="shortcut icon" href="/newimages/apps/b2c/kyobo.ICO"/>\n<link rel="stylesheet" type="text/css" href="http://image.kyobobook.co.kr/ink/css/default_ink.css" />\n<link rel="stylesheet" type="text/css" href="http://image.kyobobook.co.kr/ink/css/search.css" />\n<!--\n

In [253]:
requests.compat.urlparse("http://pythonscraping.com/pages/cookies/login.html")

ParseResult(scheme='http', netloc='pythonscraping.com', path='/pages/cookies/login.html', params='', query='', fragment='')

In [254]:
requests.compat.urljoin('http://pythonscraping.com/pages/cookies/login.html?id=1234',"process.php")

'http://pythonscraping.com/pages/cookies/process.php'

In [307]:
url = "http://pythonscraping.com/pages/cookies/login.html"
url = requests.compat.urljoin(url,"welcome.php")
data = {"username":"1234",
               "password":"password"}

In [265]:
html3 = download("post",
                url, 
               data=data)

In [259]:
html3.text

'\n<h2>Welcome to the Website!</h2>\nWhoops! You logged in wrong. Try again with any username, and the password "password"<br><a href="login.html">Log in here</a>'

In [260]:
html3.url

'http://pythonscraping.com/pages/cookies/welcome.php?serviceKey=PGlojt4q3MXsNb2WywMWZNXPnV0sMYmPnZHjdER8uUq0ceI9rGlicEnKMGOoGnFeTGeEZZI9MeJAWZ43cXokzw%3D%3D&numOfRows=10&pageNo=1&sidoName=%EC%84%9C%EC%9A%B8&searchCondition=DAILY&_returnType=JSON'

-> 쿠키, 세션이 생성이 안되서... 로그인 안된다.

In [308]:
session = requests.Session()

In [296]:
data

{'username': '1234', 'password': '1234'}

In [309]:
html = session.post(url,data)

In [304]:
html.text

'\n<h2>Welcome to the Website!</h2>\nWhoops! You logged in wrong. Try again with any username, and the password "password"<br><a href="login.html">Log in here</a>'

In [299]:
html.url

'http://pythonscraping.com/pages/cookies/welcome.php'

In [300]:
html = session.post(url)

In [301]:
html.text

'\n<h2>Welcome to the Website!</h2>\nWhoops! You logged in wrong. Try again with any username, and the password "password"<br><a href="login.html">Log in here</a>'

In [302]:
session.cookies??

In [305]:
session.cookies

<RequestsCookieJar[]>

In [284]:
html = session.post??

In [311]:
html = session.post(url,data)

In [312]:
html.text

'\n<h2>Welcome to the Website!</h2>\nYou have logged in successfully! <br><a href="profile.php">Check out your profile!</a>'

In [313]:
session.cookies

<RequestsCookieJar[Cookie(version=0, name='loggedin', value='1', port=None, port_specified=False, domain='pythonscraping.com', domain_specified=False, domain_initial_dot=False, path='/pages/cookies', path_specified=False, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={}, rfc2109=False), Cookie(version=0, name='username', value='1234', port=None, port_specified=False, domain='pythonscraping.com', domain_specified=False, domain_initial_dot=False, path='/pages/cookies', path_specified=False, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={}, rfc2109=False)]>

In [314]:
html = session.get("https://everytime.kr/393758")

In [315]:
html.text

'<!DOCTYPE html>\n<html lang="ko">\n<head>\n  <title>에브리타임</title>\n  <meta charset="utf-8">\n  <meta name="referrer" content="origin">\n  <meta http-equiv="X-UA-Compatible" content="IE=edge">\n  <meta property="fb:app_id" content="258676027581965">\n  <meta property="og:type" content="website">\n  <meta property="og:image" content="https://everytime.kr/images/og_image.png">\n  <meta property="og:url" content="https://everytime.kr/393758">\n  <meta property="og:site_name" content="에브리타임">\n  <meta property="og:title" content="에브리타임">\n  <meta property="og:description" content="전국 400개 대학을 지원하는 대학교 커뮤니티 및 시간표 서비스. 시간표 작성 및 학업 관리, 학교 생활 정보, 학교별 익명 커뮤니티 기능을 제공합니다.">\n  <meta name="description" content="전국 400개 대학을 지원하는 대학교 커뮤니티 및 시간표 서비스. 시간표 작성 및 학업 관리, 학교 생활 정보, 학교별 익명 커뮤니티 기능을 제공합니다.">\n  <meta name="keywords" content="에브리타임, 에타, everytime, 시간표, 수강신청, 강의평가, 학점계산기, 학식, 오늘의학식, 책방, 중고책, 대학생, 대학교, 대학, 대학생 시간표, 대학교 시간표, 대학생 커뮤니티, 대학교 커뮤니티, 시간표 앱, 시간표 어플">\n  <meta name="naver-site-verificat

In [316]:
html = session.get("https://lms.sunde41.net/course/2")

In [317]:
html.text

'<!DOCTYPE html>\n<html lang="ko">\n<head>\n    <meta charset="UTF-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n    <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, shrink-to-fit=no">\n    <meta name="description" content="">\n    <meta name="author" content="">\n    <title> 로그인 : 빅데이터 청년인재 고려대학교 과정 학습관리시스템 </title>\n    <link href="/static/vendors/login.css" rel="stylesheet">\n    <link href="/static/vendors/bootstrap.css" rel="stylesheet">\n    <link href="/static/vendors/style.css?v=1.7" rel="stylesheet">\n    <link rel="icon" href="/static/favicon/cropped-m_logo-32x32.png" sizes="32x32">\n    <link rel="icon" href="/static/favicon/cropped-m_logo-192x192.png" sizes="192x192">\n    <link rel="apple-touch-icon-precomposed" href="/static/favicon/cropped-m_logo-180x180.png">\n    <meta name="msapplication-TileImage" content="/static/favicon/cropped-m_logo-270x270.png">\n    <link rel="icon" href="http://bigjob.dbguide.net/w